In [1]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:95% !important; }</style>"))
display(HTML("<style>.input_area pre {font-family: Consolas; font-size: 16pt; line-height: 140%;}</style>"))
display(HTML("<style>.output_area pre {font-family: Consolas; font-size: 16pt; line-height: 140%;}</style>"))

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

import pandas as pd
import numpy as np
import os
import glob
import datetime
import shutil
from collections import defaultdict, Counter
from copy import deepcopy
import networkx as nx
import matplotlib.pyplot as plt
from matplotlib import font_manager, rc
font_name = font_manager.FontProperties(fname="c:/Windows/Fonts/malgun.ttf").get_name()
plt.rc('font', family=font_name)

In [2]:
def add_ic50_and_col_norm(file_name):
    cell = file_name.split('_')[-2]
    
    ic50_df=pd.read_table('./result/GDSC_IC50s_for_common_drugs.txt', index_col=0, sep='\t',engine='python')
    ic50_ds=ic50_df[cell]
    ic50_ds=ic50_ds[~ic50_ds.isna()]
    ic50_dic=ic50_ds.to_dict()

    
    TF_activity_df=pd.read_table(file_name, sep='\t',engine='python')
    #print(TF_activity_df.head())
    
    # get effect info
    TF_activity_df['compound']=TF_activity_df['compound'].map(ic50_dic)
    TF_activity_df=TF_activity_df[~TF_activity_df['compound'].isna()]
    TF_activity_df.rename(columns={'compound':'ic50'},inplace=True)
    
    ic50_ds=TF_activity_df['ic50']
    dose_ds=TF_activity_df['dose']
    diff_ds=dose_ds-ic50_ds
    TF_activity_df['effect']=np.where(diff_ds>=0, 1, 0)
    TF_activity_df.drop(['ic50','dose'],axis=1,inplace=True)
    
    tot=len(TF_activity_df['effect'])
    eff=np.sum(TF_activity_df['effect']==1)
    if (tot<400) or ((eff/tot)<=0.15):
        return None
    
    print('######### PASS #######', cell)

    ## 각 TF_activity row normalize 수행하기
    for ind in TF_activity_df.index:
        values=TF_activity_df.loc[ind].iloc[:-1]
        
        x_min=values.min()
        x_max=values.max()
        
        norm1_ds=(values-x_min)/(x_max-x_min+1e-7)
        norm2_ds=(norm1_ds-norm1_ds.mean())/norm1_ds.std()
        norm2_ds.loc['effect']=TF_activity_df.loc[ind,'effect']
        
        TF_activity_df.loc[ind]=norm2_ds
        
    out_file_name=file_name.replace("comp_dose", "effect").replace("CompDose",'Effect')
    TF_activity_df.to_csv(out_file_name,sep='\t',index=False)
        
file_names=glob.glob('result/TF_activity/comp_dose/*.txt')
for file_name in file_names:
    add_ic50_and_col_norm(file_name)

######### PASS ####### A375
######### PASS ####### HT29
